## ACY Deep Learning Training Ground #1

Welcome to Deep Learning Training Ground! Here we are going to train a simple Deep Neural Network which can recognize Myanmar Handwritten Digits. First, make sure that your Colab Runtime is using GPU. In order to do so:


* Go to Edit > Notebook Settings
* Select **GPU** in Hardware Accelerator dropdown
* Click on "Save"



### Preparation

Install necessary libraries.


*   **Augmentor** is a data agumentation library. As a matter of fact, we don't need to perform prelimenary data augmentation since we can implement data augmentation when the data batches are loaded for training. The purpose of having a separate data augmentation step is for demonstrative purpose.
*   **fastai** is the deep learning wrapper library which makes training neural networks extremely easy. **fastai** is a good place to start getting familiar with Deep Learning.
* **torch** == `PyTorch` is a deep learning framework by Facebook. It is one of the most popular deep learning frameworks out there. Tensorflow is another extremely popular choice. **fastai** uses `PyTorch` as a backend therefore, we have to install `PyTorch`.

* Additionally, we also install `unzip` software to unzip files on Linux.



In [ ]:
%%sh

pip install -q Augmentor
pip install -q fastai
pip install -q "torch==1.4" "torchvision==0.5.0"
sudo apt-get -q install unzip

I have create Myanmar Handwritten Digits by drawing them in MSPaint with different brushes. First, let's download that zip file and check out the images in there. We are going to train a neural network which can recognize Myanmar Hand written Digits from this dataset.

Download the zip file.

In [ ]:
!wget https://github.com/ArkAung/dl_acy_workshop/raw/master/datasets/numbers/numbers.zip

In [ ]:
!unzip -qq numbers.zip

## Augment Data

As you are aware of the fact that there only are a few images for each digit, and deep learning is such a data hungry technique, we are going to **augment** the data. (Data augmentation = data ကိုတိုးပွားလာအောင်လုပ်သည်)

Using `Augmentor` library, we are going to create different variants of our original data by applying transformations such as:


* Random rotations
* Random distortions
* Random shearing

<img src="https://raw.githubusercontent.com/mdbloice/AugmentorFiles/master/UsageGuide/rotate_aug_b.png" alt="Rotation Example" width="100"/>
<img src="https://cloud.githubusercontent.com/assets/16042756/23697283/802698a6-03e7-11e7-94b7-f0b61977ef33.gif" alt="Random Distortion" width="100"/>
<img src="https://raw.githubusercontent.com/mdbloice/AugmentorFiles/master/UsageGuide/shear_x_aug.png" alt="Shearing along x-axis" width="100"/>
<img src="https://raw.githubusercontent.com/mdbloice/AugmentorFiles/master/UsageGuide/shear_y_aug.png" alt="Shearing along y-axis" width="100"/>

Ref: [Augmentor Repo](https://github.com/mdbloice/Augmentor)

In [ ]:
import Augmentor
p = Augmentor.Pipeline("numbers") # Since our images are in "numbers" directory

In [ ]:
p.resize(probability=1, width=100, height=100)
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.random_distortion(probability=0.9, grid_width=3, grid_height=3, magnitude=8)
p.shear(probability=0.3, max_shear_left=5, max_shear_right=5)

In [ ]:
p.process()
p.sample(10000)

Augmented images are saved in a folder `output`. We are going to move this to a directory called `train`.

In [ ]:
!mv numbers/output train

## Prepare data for training

In [ ]:
from fastai.vision import * 

We have folders containing image files of each digit in a parent folder called "train". In oroder to train a neural network, we have to prepare those images in a format which the neural network can consume.

In [ ]:
path="train"
np.random.seed(42)
data = ImageDataBunch.from_folder(path, train=path,valid_pct=0.2,
                                  ds_tfms=get_transforms(do_flip=False, max_rotate=2.0,
                                                         max_zoom=1, max_warp=0.0,
                                                         p_affine=0.0),
                                  size=100, num_workers=4)

## Visualize Training Data

In [ ]:
data.show_batch()

## Create Deep Neural Network to Train on Data

We are going to use a neural network architecture called [AlexNet](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf).
We are going to measure how well our neural network is doing by two metrics, Error Rate and Accuracy.

In [ ]:
from fastai.metrics import error_rate
learn = cnn_learner(data, models.alexnet, metrics=[error_rate, accuracy],
                    model_dir="/content/model/")

## Start Training

Since we are using Google CoLab with GPU, we can specify 'cuda' here. If you only have a CPU available (for example, on your machine), you would have to specify cpu here.

Then, start training the neural network for 4 epochs using [Cyclical Learning Rates](https://arxiv.org/pdf/1506.01186.pdf).

In [ ]:
defaults.device = torch.device('cuda')
learn.fit_one_cycle(4)

## Understanding the results

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(9, figsize=(6,6))

In [ ]:
interp.plot_confusion_matrix()

## Export the Trained Model

In [ ]:
learn.export()

## Loading saved model to make prediction on a single image

In [ ]:
from fastai.vision import *

In [ ]:
learn = load_learner('train', 'export.pkl')
im = open_image('numbers/၃/Annotation 2020-06-27 114134.png')
preds_class, preds_idx, preds_output = learn.predict(im)
class_idx = preds_class.data.item()
mapping = {v:k for k,v in learn.data.c2i.items()}

In [ ]:
im.show()
print("Result is: {}".format(mapping[class_idx]))

## Setting up a webserver to predict hand written numbers

Install necessary libraries for Flask server

In [ ]:
%%sh

pip install -q flask-ngrok
pip install -q flask==0.12.2

Prepare source for webserver

In [ ]:
%%sh
git clone https://github.com/ArkAung/dl_acy_workshop.git
mv dl_acy_workshop/webserver.py .
mv dl_acy_workshop/static .
mv dl_acy_workshop/templates .

## Run the Webserver

In [ ]:
!python webserver.py --type "digits"